In [ ]:
"""
    Pappus' creativity is due to him (implicitly) distinguishing between numerical identity and conceptual identity
    in step 1 and applying this distinction to the triangle ABC to take advantage of Euclid I.4.
"""

!python --version

## install and import packages
# !pip install nbimporter # to import notebooks
# !pip install rdflib

## import modules
import copy
# import glob
import json
# import os
# import rdflib
import re
import requests
import typing # to add advanced type hinting
import urllib

## import custom modules
import functions_find_conceptual_space
import functions_heuristics
import functions_utils
import sparql_classes
import sparql_queries

# !pwd

# PROOFS OF EUCLID I.5

In [ ]:
proof_step = "<https://www.foom.com/pappus_proofPappus#00000000000000000150>"
selected_datastore:str = "/datastores/foom_pappus/sparql"
conceptual_space_before_proof_step = functions_find_conceptual_space.find_conceptual_space_before_proof_step(proof_step, selected_datastore)
conceptual_space_of_proof_step = functions_find_conceptual_space.find_conceptual_space_of_proof_step(proof_step, selected_datastore)
diff_conceptual_space = functions_utils.diff_conceptual_spaces(
    conceptual_space_before_proof_step,
    conceptual_space_of_proof_step)

In [ ]:
diff_conceptual_space

In [ ]:
proof_step = "<https://www.foom.com/pappus_proofPappus#00000000000000000150>"
proof_iri = "<https://www.foom.com/pappus_proofPappus#00000000000000000131>"
claim_iri = "<http://www.foom.com/mathematical_concepts#00000000000000000115>"
selected_datastore:str = "/datastores/foom_pappus/sparql"
functions_heuristics.find_goal_of_proof(proof_iri, selected_datastore)
functions_heuristics.find_last_step_of_proof(proof_iri, selected_datastore)

functions_find_conceptual_space.find_conceptual_space_of_claim(claim_iri, selected_datastore)